# Explore network statistics

In this notebook we count for number of nodes and edges for each node and edge type and generate statistics and visualizations for the PINetwork generated in "Sinai2024.ipynb".

In [ ]:
# Import statements
import pandas as pd
import os
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
import random
from tqdm import tqdm
import numpy as np
import webbrowser

Here we generate counts for each of the nodes and edges of the network.

# Protocol 1 - 21

In [ ]:
for file in os.listdir("./Network/"):
    if file.endswith(".csv"):
        df = pd.read_csv("./Network/" + file)
        print(file, len(df))


Get total network nodes and edges.

In [ ]:
# Create variables for node and edge counts.
node_count = 0
edge_count = 0 

# Iterate over node files to get node counts.
for file in os.listdir("./Network/"):
    if file.endswith(".nodes.csv"):
        df = pd.read_csv("./Network/" + file)
        node_count += len(df)
    elif file.endswith(".edges.csv"):
        df = pd.read_csv("./Network/" + file)
        edge_count += len(df)

# Print out node and edge counts.
print(f"Total Node Count: {node_count}")
print(f"Total Edge Count: {edge_count}")

# Protocol 22

Now let us create plots for the node connectivity distribution. This visualizes the distribution of the number of neighbors all the nodes in the graph have. We do this visualization for each node type to understand the underlying structure of the graph and to ensure that nothing is wrong with the network itself.

In [ ]:
# Initialize structures
node_type_map = {}
node_degrees = defaultdict(int)

network_path = "./Network/"

# Step 1: Map node IDs to their types
for file in os.listdir(network_path):
    if file.endswith(".nodes.csv"):
        df = pd.read_csv(os.path.join(network_path, file), usecols=['id'])
        node_type = file.replace(".nodes.csv", "")
        for node_id in df['id']:
            node_type_map[node_id] = node_type

# Step 2: Count degrees
seen_edges = set()
for file in os.listdir(network_path):
    if file.endswith(".edges.csv"):
        df = pd.read_csv(os.path.join(network_path, file), usecols=['source', 'target'])
        for src, tgt in zip(df['source'], df['target']):
            edge = tuple(sorted((src, tgt)))
            if edge not in seen_edges:
                seen_edges.add(edge)
                node_degrees[edge[0]] += 1
                node_degrees[edge[1]] += 1

# Step 3: Aggregate degree by type
degree_by_type = defaultdict(list)
for node_id, degree in node_degrees.items():
    node_type = node_type_map.get(node_id, "Unknown")
    degree_by_type[node_type].append(degree)

# Step 4: Plot 2x2 subplots
node_types = list(degree_by_type.keys())
num_plots = len(node_types)
rows, cols = 2, 2

fig, axes = plt.subplots(rows, cols, figsize=(12, 10))
axes = axes.flatten()

for i, node_type in enumerate(node_types[:4]):
    ax = axes[i]
    degrees = degree_by_type[node_type]


    ax.hist(
        degrees,
        bins=20,  # +2 to include max degree
        edgecolor='black',
        color='gray',
        align='mid',
        rwidth=1.0  # no padding between bars
    )
    #ax.set_xlim(min_deg - 0.5, max_deg + 0.5)  # optional: centers bars better
    ax.set_title(f"{node_type} Neighbors")
    if node_type == "Authors":
        ax.set_title("PIs Neighbors")
    ax.set_xlabel("Neighbors")
    ax.set_ylabel(f"{node_type}")
    ax.set_yscale("log")

# Hide unused subplots
for j in range(len(node_types), rows * cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


<small> Figure 2A. The neighbor distribution of each node type for the generated network. </small>

Now let us calculate the average of the shortest paths. This means that it finds all paths to all other nodes and then takes the mean of that path length. We do this for each node type to understand how certain node types are connected to the rest of the graph.

In [ ]:
network_path = "./Network/"
G = nx.Graph()
node_type_map = {}

# Load node types
for file in os.listdir(network_path):
    if file.endswith(".nodes.csv"):
        node_type = file.replace(".nodes.csv", "")
        df = pd.read_csv(os.path.join(network_path, file), usecols=['id'])
        for node_id in df['id']:
            node_type_map[node_id] = node_type

# Load edges
for file in os.listdir(network_path):
    if file.endswith(".edges.csv"):
        df = pd.read_csv(os.path.join(network_path, file), usecols=['source', 'target'])
        G.add_edges_from(zip(df['source'], df['target']))

# Compute all-pairs shortest paths
all_lengths = []
type_lengths = defaultdict(list)

for source, targets in tqdm(nx.all_pairs_shortest_path_length(G), total=len(G), desc="Computing paths"):
    source_type = node_type_map.get(source, "Unknown")
    for target, dist in targets.items():
        if source != target:
            all_lengths.append(dist)
            if node_type_map.get(target, "Unknown") == source_type:
                type_lengths[source_type].append(dist)

# Print results
print(f"\nTotal Average Shortest Path Length: {np.mean(all_lengths):.4f}")
for ntype, lengths in type_lengths.items():
    print(f"Avg Path Length within node type '{ntype}': {np.mean(lengths):.4f}")



In [ ]:
import random
samp = 1000000
sampled_lengths_by_type = {
    ntype: random.sample(lengths, min(samp, len(lengths)))
    for ntype, lengths in type_lengths.items()
}
# Create subplots for each node type
node_types = list(sampled_lengths_by_type.keys())
ncols = 2
nrows = (len(node_types) + 1) // ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 5 * nrows))
axes = axes.flatten()

for i, ntype in enumerate(node_types):
    ax = axes[i]
    ax.hist(sampled_lengths_by_type[ntype], bins=30, edgecolor='black', color='gray', align='mid')
    ax.set_title(f"Shortest Path Distribution from {ntype} nodes")
    if ntype == "Authors":
        ax.set_title(f"Shortest Path Distribution from PIs")
    ax.set_xlabel("Path Length")
    ax.set_ylabel("Paths")

# Remove unused subplots
for j in range(len(node_types), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle(f"Shortest Path Distribution n={samp}", fontsize=20)
plt.tight_layout()
plt.show()

<small> Figure 2B. A sampling of the shortest path distribution from each node type. The distribution is sampled from the true distribution using 10^6 data points. </small>

Now let us compute all of the top node types by each edge type (i.e. top Author by number of coauthors, top Author by number of publications...)

In [ ]:
network_path = "./Network/"
node_labels = {}
node_types = {}

# Step 1: Load node labels (or titles) and types
for file in os.listdir(network_path):
    if file.endswith(".nodes.csv"):
        node_type = file.replace(".nodes.csv", "")
        df = pd.read_csv(os.path.join(network_path, file))

        # Use "Title" for specific node types, otherwise "label"
        if node_type == "Publications" and "Journal" in df.columns:
            label_col = "Journal"
        elif node_type == "Awards" and "Title" in df.columns:
            label_col = "Title"
        else:
            label_col = "label"

        df['id'] = df['id'].astype(str)
        df[label_col] = df[label_col].astype(str)

        for node_id, label in zip(df['id'], df[label_col]):
            if node_type == "Publications":
                full_label = f"PMID: {node_id}, {label}"
            else:
                full_label = label
            node_labels[node_id] = full_label
            node_types[node_id] = node_type

# Step 2: Compute degrees per edge type, only when node_type matches the left side
edge_type_degrees = defaultdict(lambda: defaultdict(list))  # node_type -> edge_type -> [(label, degree)]


length_top_graph = {("Authors", "Authors"): 20,
                    ("Authors", "Publications"): 20,
                    ("Awards", "Publications"): 20}
for file in os.listdir(network_path):
    if file.endswith(".edges.csv"):
        edge_type_full = file.replace(".edges.csv", "")
        parts = edge_type_full.split('.')
        if len(parts) < 3:
            continue

        type1, relation, type2 = parts[0], parts[1], parts[2]

        # Only process edge if node_type is in first position
        if type1 not in node_types.values():
            continue

        df = pd.read_csv(os.path.join(network_path, file), usecols=['source', 'target'])
        df['source'] = df['source'].astype(str)
        df['target'] = df['target'].astype(str)

        G = nx.Graph()
        G.add_edges_from(zip(df['source'], df['target']))

        for node, deg in G.degree():
            if node_types.get(node) == type1:
                label = node_labels.get(node, node)
                edge_type_degrees[type1][edge_type_full].append((label, deg))

# Step 3: Plot top nodes
for node_type, edge_map in edge_type_degrees.items():
    edge_types = list(edge_map.keys())
    ncols = 1
    nrows = len(edge_types)

    if node_type == "Awards":
        nrows -= 1
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 4 * nrows))
    if nrows == 1:
        axes = [axes]

    for i, edge_type in enumerate(edge_types):
        
        top = sorted(edge_map[edge_type], key=lambda x: x[1], reverse=True)[:length_top_graph.get(
            (node_type, edge_type.split(".")[2]), 10
        )]
        raw_labels, degrees = zip(*top)

        # Cap all labels at 60 characters
        labels = []
        for label in raw_labels:
            label = label.replace("N/A,", "").replace("N/A", "").strip()
            label = ' '.join(label.split())  # remove extra whitespace
            label = label[:60]
            if len(label) == 60:
                label += "..."
            labels.append(label)

        relation = edge_type.split('.')[2]  # Use third part as relation
        if node_type == "Awards" and relation == "Authors":
            continue
        ax = axes[i]
        ax.barh(labels[::-1], degrees[::-1], color='gray', edgecolor='black')
        ax.set_title(f"Top {node_type} by {relation}")
        if node_type == "Authors" or relation == "Authors":
            if node_type == "Authors" and relation == "Authors":
                ax.set_title(f"Top PIs by PI Coauthors")
            elif relation == "Authors":
                ax.set_title(f"Top {node_type} by PIs")
            elif node_type == "Authors":
                ax.set_title(f"Top PIs by {relation}")


        ax.set_xlabel(f"{relation}")

    plt.subplots_adjust(top=1.1)
    plt.tight_layout()
    plt.show()


<small>Figure 3. The top nodes for each node type by each edge type. For example, we find the top PIs by their number of publications or the number of coauthor PIs. For the plots either the top 10 or the top 20 are displayed.</small>